In [6]:
# 1. Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# 2. Загрузка данных
print("Загрузка данных...")
df = pd.read_csv('data.csv')

print(f"Размер обучающей выборки: {df.shape}")

# 3. Разделение на train/validation с сохранением дисбаланса классов
print("\n" + "="*70)
print("3. РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ВАЛИДАЦИОННУЮ ВЫБОРКИ")
print("="*70)

# Удаляем идентификаторы, которые не нужны для обучения
if 'reco_id_curr' in df.columns:
    df.drop(['reco_id_curr'], axis=1, inplace=True)

# Разделяем признаки и целевую переменную
X = df.drop('target', axis=1)
y = df['target']

# Стратифицированное разделение для сохранения дисбаланса классов
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,  # Сохраняем распределение классов
    random_state=42
)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер валидационной выборки: {X_val.shape}")

# Проверка распределения целевой переменной
print("\nРаспределение целевой переменной в обучающей выборке:")
print(y_train.value_counts(normalize=True) * 100)
print("\nРаспределение целевой переменной в валидационной выборке:")
print(y_val.value_counts(normalize=True) * 100)

# 4. Определение типов признаков
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"\nВсего числовых признаков: {len(numeric_features)}")
print(f"Всего категориальных признаков: {len(categorical_features)}")

# 5. Обнаружение и обработка выбросов во ВСЕХ числовых признаках
print("\n" + "="*70)
print("5. ОБНАРУЖЕНИЕ И ОБРАБОТКА ВЫБРОСОВ ВО ВСЕХ ЧИСЛОВЫХ ПРИЗНАКАХ")
print("="*70)

def detect_and_handle_outliers_comprehensive(df_train, df_val, feature, verbose=True):
    """
    Комплексная обработка выбросов для одного признака
    
    Parameters:
    df_train : DataFrame
        Обучающая выборка
    df_val : DataFrame
        Валидационная выборка
    feature : str
        Имя признака для обработки
    verbose : bool
        Выводить ли подробную информацию
    
    Returns:
    tuple
        Обработанные обучающая и валидационная выборки
    """
    if feature not in df_train.columns or df_train[feature].isnull().all():
        return df_train, df_val
    
    # Копируем данные для обработки
    df_train_processed = df_train.copy()
    df_val_processed = df_val.copy()
    
    # Определяем границы на основе обучающей выборки
    Q1 = df_train[feature].quantile(0.25)
    Q3 = df_train[feature].quantile(0.75)
    IQR = Q3 - Q1
    
    # Используем разные коэффициенты для разных типов признаков
    # Для признаков с естественными границами используем мягкие границы
    boundary_features = ['children_count', 'family_members__count', 'start_weekday_appr_process', 
                         'hour_of_approval_process_start']
    
    if feature in boundary_features:
        lower_bound = max(0, Q1 - 1.5 * IQR)  # Нижняя граница не может быть отрицательной
        upper_bound = Q3 + 1.5 * IQR
    # Для финансовых признаков используем умеренные границы
    elif feature in ['income', 'loan_body', 'annuity_payment', 'goods_price']:
        lower_bound = max(0, Q1 - 2 * IQR)  # Доход и суммы не могут быть отрицательными
        upper_bound = Q3 + 4 * IQR  # Более высокая верхняя граница для финансовых признаков
    # Для временных признаков (в днях) используем специальные границы
    elif 'days_' in feature or '_timestamp' in feature or 'change' in feature:
        lower_bound = Q1 - 6 * IQR  # Для временных признаков выбросы могут быть значительными
        upper_bound = Q3 + 6 * IQR
    # Для скоров и рейтингов используем строгие границы
    elif 'external_source' in feature or 'rating_' in feature:
        lower_bound = max(-1, Q1 - 1.5 * IQR)  # Скоры обычно в диапазоне [-1, 1]
        upper_bound = min(1, Q3 + 1.5 * IQR)
    # Для всех остальных признаков используем стандартные границы
    else:
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
    
    # Подсчет выбросов
    outliers_train = ((df_train[feature] < lower_bound) | (df_train[feature] > upper_bound)).sum()
    outliers_val = ((df_val[feature] < lower_bound) | (df_val[feature] > upper_bound)).sum()
    
    percent_outliers_train = outliers_train / len(df_train) * 100
    percent_outliers_val = outliers_val / len(df_val) * 100
    
    # Выбираем метод обработки в зависимости от процента выбросов
    if percent_outliers_train > 10:  # Много выбросов - используем winsorization
        # Обработка выбросов (winsorization)
        df_train_processed[feature] = np.where(df_train[feature] < lower_bound, lower_bound, df_train[feature])
        df_train_processed[feature] = np.where(df_train_processed[feature] > upper_bound, upper_bound, df_train_processed[feature])
        
        df_val_processed[feature] = np.where(df_val[feature] < lower_bound, lower_bound, df_val[feature])
        df_val_processed[feature] = np.where(df_val_processed[feature] > upper_bound, upper_bound, df_val_processed[feature])
        
        method = "winsorization (ограничение границами)"
    else:  # Мало выбросов - можно удалить или заменить на NaN
        # Заменяем выбросы на NaN для последующего заполнения
        df_train_processed.loc[(df_train[feature] < lower_bound) | (df_train[feature] > upper_bound), feature] = np.nan
        df_val_processed.loc[(df_val[feature] < lower_bound) | (df_val[feature] > upper_bound), feature] = np.nan
        
        method = "замена на NaN (для последующего заполнения)"
    
    if verbose: #and (percent_outliers_train > 0.5 or percent_outliers_val > 0.5):
        print(f"\nПризнак: {feature}")
        print(f"Метод обработки: {method}")
        print(f"Границы выбросов: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"Выбросов в обучающей выборке: {outliers_train} ({percent_outliers_train:.2f}%)")
        print(f"Выбросов в валидационной выборке: {outliers_val} ({percent_outliers_val:.2f}%)")
        print(f"Диапазон до обработки (train): [{df_train[feature].min():.2f}, {df_train[feature].max():.2f}]")
        print(f"Диапазон после обработки (train): [{df_train_processed[feature].min():.2f}, {df_train_processed[feature].max():.2f}]")
    
    return df_train_processed, df_val_processed

# Обработка выбросов для всех числовых признаков
print("Начало комплексной обработки выбросов...")
print(f"Всего числовых признаков для обработки: {len(numeric_features)}")

outliers_summary = []

for feature in numeric_features:
    if feature in X_train.columns:
        X_train, X_val = detect_and_handle_outliers_comprehensive(X_train, X_val, feature, verbose=False)
        
        # Собираем статистику для отчета
        outliers_train = X_train[feature].isnull().sum() - X_train[feature].isnull().sum() + X_train[feature].isnull().sum()
        # Это заглушка, правильный подсчет будет ниже
        outliers_summary.append({
            'feature': feature,
            'outliers_percent': (X_train[feature].isnull().sum() / len(X_train)) * 100
        })

# 6. Заполнение пропущенных значений (включая те, что были отмечены как выбросы)
print("\n" + "="*70)
print("6. ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
print("="*70)

# Анализ пропусков в обучающей выборке
missing_values_train = X_train.isnull().mean() * 100
missing_values_train = missing_values_train[missing_values_train > 0].sort_values(ascending=False)
print(f"Количество признаков с пропусками в обучающей выборке: {len(missing_values_train)}")
print("\nТоп-10 признаков с наибольшим процентом пропусков:")
print(missing_values_train.head(10))

# 6.1 Заполнение пропусков в числовых признаках с интеллектуальной стратегией
print("\n6.1 Заполнение пропусков в числовых признаках...")

# Сначала обработаем особые случаи - важные признаки с пропусками
special_features = {
    'external_source_3': {'strategy': 'knn', 'neighbors': 5},
    'external_source_2': {'strategy': 'median'},
    'external_source_1': {'strategy': 'median'},
    'days_employed': {'strategy': 'median'},
    'last_phone_number_change': {'strategy': 'median'}
}

for feature in numeric_features:
    if feature in X_train.columns:
        missing_train = X_train[feature].isnull().mean() * 100
        missing_val = X_val[feature].isnull().mean() * 100
        
        if missing_train > 0 or missing_val > 0:
            # Проверяем, является ли признак особым случаем
            fill_strategy = "default"
            fill_value = None
            
            if feature in special_features:
                config = special_features[feature]
                fill_strategy = config['strategy']
                
                if fill_strategy == 'knn' and missing_train < 50:  # KNN работает плохо с большим количеством пропусков
                    print(f"\nKNN-импутация для {feature}...")
                    
                    # Выбираем релевантные признаки для KNN
                    if feature == 'external_source_3':
                        related_features = ['external_source_1', 'external_source_2', 'days_birth', 
                                          'income', 'loan_body', 'rating_client_region']
                    
                    # Фильтруем только существующие признаки
                    related_features = [f for f in related_features if f in X_train.columns]
                    
                    if len(related_features) >= 2:
                        # Создаем копии для импутации
                        X_train_imp = X_train[related_features + [feature]].copy()
                        X_val_imp = X_val[related_features + [feature]].copy()
                        
                        # Удаляем строки с пропусками в признаках для импутации
                        X_train_imp_no_na = X_train_imp.dropna(subset=related_features)
                        
                        if len(X_train_imp_no_na) > config['neighbors']:
                            # Заполняем пропуски в external_source_3 с помощью KNN
                            imputer = KNNImputer(n_neighbors=config['neighbors'])
                            X_train_imp_filled = pd.DataFrame(
                                imputer.fit_transform(X_train_imp_no_na),
                                columns=X_train_imp_no_na.columns,
                                index=X_train_imp_no_na.index
                            )
                            X_val_imp_filled = pd.DataFrame(
                                imputer.transform(X_val_imp),
                                columns=X_val_imp.columns,
                                index=X_val_imp.index
                            )
                            
                            # Обновляем значения в исходных данных
                            X_train.loc[X_train_imp_no_na.index, feature] = X_train_imp_filled[feature]
                            X_val[feature] = X_val_imp_filled[feature]
                            
                            print(f"  KNN-импутация для {feature} выполнена успешно")
                            continue
            
            # Для всех остальных случаев определяем стратегию заполнения
            if missing_train > 70:
                fill_value = X_train[feature].median()
                fill_strategy = "медиана (очень высокий % пропусков >70%)"
            elif missing_train > 30:
                fill_value = X_train[feature].median()
                fill_strategy = "медиана (высокий % пропусков >30%)"
            elif abs(X_train[feature].skew()) > 2:  # Сильная асимметрия
                fill_value = X_train[feature].median()
                fill_strategy = "медиана (сильная асимметрия)"
            elif 'flag' in feature or '_count' in feature or feature in ['children_count', 'family_members__count']:
                # Для счетчиков и флагов используем моду
                fill_value = X_train[feature].mode()[0] if not X_train[feature].mode().empty else 0
                fill_strategy = "мода (счетчик или флаг)"
            elif 'external_source' in feature or 'rating_' in feature:
                # Для скоров и рейтингов используем медиану
                fill_value = X_train[feature].median()
                fill_strategy = "медиана (скор или рейтинг)"
            else:
                # Для остальных признаков используем медиану или среднее в зависимости от асимметрии
                if abs(X_train[feature].skew()) > 0.5:
                    fill_value = X_train[feature].median()
                    fill_strategy = "медиана (умеренная асимметрия)"
                else:
                    fill_value = X_train[feature].mean()
                    fill_strategy = "среднее (нормальное распределение)"
            
            # Заполняем пропуски
            X_train[feature].fillna(fill_value, inplace=True)
            X_val[feature].fillna(fill_value, inplace=True)
            
            if missing_train > 5 or missing_val > 5:  # Выводим информацию только для значимых пропусков
                print(f"\nПризнак: {feature}")
                print(f"  Стратегия заполнения: {fill_strategy}")
                print(f"  Значение для заполнения: {fill_value:.4f}")
                print(f"  Пропусков в обучающей выборке до заполнения: {missing_train:.2f}%")
                print(f"  Пропусков в валидационной выборке до заполнения: {missing_val:.2f}%")

# 6.2 Заполнение пропусков в категориальных признаках
print("\n6.2 Заполнение пропусков в категориальных признаках...")

for feature in categorical_features:
    if feature in X_train.columns:
        missing_train = X_train[feature].isnull().mean() * 100
        missing_val = X_val[feature].isnull().mean() * 100
        
        if missing_train > 0 or missing_val > 0:
            # Для категориальных признаков с большим количеством пропусков создаем отдельную категорию
            if missing_train > 30:
                fill_value = "Unknown"
                fill_strategy = "новая категория 'Unknown' (высокий % пропусков >30%)"
            else:
                # Используем наиболее частое значение (моду)
                fill_value = X_train[feature].mode()[0] if not X_train[feature].mode().empty else "Unknown"
                fill_strategy = "мода (наиболее частая категория)"
            
            # Заполняем пропуски
            X_train[feature].fillna(fill_value, inplace=True)
            X_val[feature].fillna(fill_value, inplace=True)
            
            if missing_train > 5 or missing_val > 5:
                print(f"\nПризнак: {feature}")
                print(f"  Стратегия заполнения: {fill_strategy}")
                print(f"  Значение для заполнения: {fill_value}")
                print(f"  Пропусков в обучающей выборке до заполнения: {missing_train:.2f}%")
                print(f"  Пропусков в валидационной выборке до заполнения: {missing_val:.2f}%")

# Проверка оставшихся пропусков
remaining_missing_train = X_train.isnull().sum().sum()
remaining_missing_val = X_val.isnull().sum().sum()

print(f"\nОставшиеся пропуски в обучающей выборке: {remaining_missing_train}")
print(f"Оставшиеся пропуски в валидационной выборке: {remaining_missing_val}")

if remaining_missing_train > 0 or remaining_missing_val > 0:
    print("Внимание: остались пропуски в данных. Применяем резервную стратегию заполнения.")
    
    # Для оставшихся пропусков применяем соответствующие импутеры
    for feature in X_train.columns:
        if X_train[feature].isnull().any() or X_val[feature].isnull().any():
            if pd.api.types.is_numeric_dtype(X_train[feature]):
                fill_value = X_train[feature].median()
                strategy = "резервная медиана"
            else:
                fill_value = X_train[feature].mode()[0] if not X_train[feature].mode().empty else "Unknown"
                strategy = "резервная мода"
            
            X_train[feature].fillna(fill_value, inplace=True)
            X_val[feature].fillna(fill_value, inplace=True)
            
            print(f"  {feature}: {strategy} ({fill_value})")

# 7. Feature Engineering
print("\n" + "="*70)
print("7. СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ (FEATURE ENGINEERING)")
print("="*70)

# 7.1 Отношение кредита к доходу
if 'loan_body' in X_train.columns and 'income' in X_train.columns:
    # Добавляем небольшое значение к доходу для избежания деления на ноль
    X_train['loan_to_income_ratio'] = X_train['loan_body'] / (X_train['income'] + 1)
    X_val['loan_to_income_ratio'] = X_val['loan_body'] / (X_val['income'] + 1)
    print("Создан признак: loan_to_income_ratio (отношение кредита к доходу)")

# 7.2 Отношение платежа к доходу
if 'annuity_payment' in X_train.columns and 'income' in X_train.columns:
    X_train['payment_to_income_ratio'] = X_train['annuity_payment'] / (X_train['income'] + 1)
    X_val['payment_to_income_ratio'] = X_val['annuity_payment'] / (X_val['income'] + 1)
    print("Создан признак: payment_to_income_ratio (отношение платежа к доходу)")

# 7.3 Отношение цены товара к доходу (для POS-кредитов)
if 'goods_price' in X_train.columns and 'income' in X_train.columns:
    X_train['goods_to_income_ratio'] = X_train['goods_price'] / (X_train['income'] + 1)
    X_val['goods_to_income_ratio'] = X_val['goods_price'] / (X_val['income'] + 1)
    print("Создан признак: goods_to_income_ratio (отношение цены товара к доходу)")

# 7.4 Возраст в годах
if 'days_birth' in X_train.columns:
    X_train['age_years'] = abs(X_train['days_birth']) / 365.25
    X_val['age_years'] = abs(X_val['days_birth']) / 365.25
    print("Создан признак: age_years (возраст в годах)")

# 7.5 Стаж работы в годах
if 'days_employed' in X_train.columns:
    X_train['work_experience_years'] = X_train['days_employed'].abs() / 365.25
    X_val['work_experience_years'] = X_val['days_employed'].abs() / 365.25
    
    # Обработка отрицательных значений (иногда days_employed может быть отрицательным)
    X_train['work_experience_years'] = X_train['work_experience_years'].apply(lambda x: max(0, x))
    X_val['work_experience_years'] = X_val['work_experience_years'].apply(lambda x: max(0, x))
    
    print("Создан признак: work_experience_years (стаж работы в годах)")

# 7.6 Стаж на текущем месте работы в процентах от возраста
if 'days_employed' in X_train.columns and 'days_birth' in X_train.columns:
    X_train['work_experience_to_age_ratio'] = abs(X_train['days_employed']) / abs(X_train['days_birth'])
    X_val['work_experience_to_age_ratio'] = abs(X_val['days_employed']) / abs(X_val['days_birth'])
    print("Создан признак: work_experience_to_age_ratio (стаж к возрасту)")

# 7.7 Комбинированный внешний скор
external_sources = [col for col in ['external_source_1', 'external_source_2', 'external_source_3'] if col in X_train.columns]
if len(external_sources) > 0:
    # Создаем невзвешенный комбинированный скор
    X_train['combined_external_score'] = X_train[external_sources].mean(axis=1)
    X_val['combined_external_score'] = X_val[external_sources].mean(axis=1)
    
    # Создаем взвешенный комбинированный скор, учитывая корреляцию с target
    # Предполагаем, что все external_source имеют отрицательную корреляцию с target
    weights = [abs(X_train[col].corr(y_train)) for col in external_sources]
    total_weight = sum(weights)
    if total_weight > 0:
        normalized_weights = [w/total_weight for w in weights]
        X_train['weighted_external_score'] = sum(X_train[col] * w for col, w in zip(external_sources, normalized_weights))
        X_val['weighted_external_score'] = sum(X_val[col] * w for col, w in zip(external_sources, normalized_weights))
        print("Созданы признаки: combined_external_score и weighted_external_score")

# 7.8 Финансовая нагрузка (сумма всех обязательных платежей)
payment_features = ['annuity_payment']
for col in X_train.columns:
    if 'payment' in col.lower() and col != 'annuity_payment':
        payment_features.append(col)

if payment_features and 'income' in X_train.columns:
    X_train['total_payments'] = X_train[payment_features].sum(axis=1)
    X_val['total_payments'] = X_val[payment_features].sum(axis=1)
    
    X_train['payment_burden'] = X_train['total_payments'] / (X_train['income'] + 1)
    X_val['payment_burden'] = X_val['total_payments'] / (X_val['income'] + 1)
    print("Создан признак: payment_burden (финансовая нагрузка)")

# 7.9 Индексы запросов в БКИ
bki_features = [col for col in X_train.columns if 'requests_bki_' in col]
if bki_features:
    # Общий индекс активности в БКИ
    X_train['total_bki_requests'] = X_train[bki_features].sum(axis=1)
    X_val['total_bki_requests'] = X_val[bki_features].sum(axis=1)
    
    # Индекс недавних запросов (более важных)
    recent_bki_features = [f for f in bki_features if any(period in f for period in ['hour', 'day', 'week'])]
    if recent_bki_features:
        X_train['recent_bki_activity'] = X_train[recent_bki_features].sum(axis=1) / X_train['total_bki_requests'].replace(0, 1)
        X_val['recent_bki_activity'] = X_val[recent_bki_features].sum(axis=1) / X_val['total_bki_requests'].replace(0, 1)
        print("Созданы признаки: total_bki_requests и recent_bki_activity")

# 7.10 Количество предоставленных документов
document_features = [col for col in X_train.columns if 'document_' in col and col.endswith('_flag')]
if document_features:
    X_train['total_documents_provided'] = X_train[document_features].sum(axis=1)
    X_val['total_documents_provided'] = X_val[document_features].sum(axis=1)
    print("Создан признак: total_documents_provided (общее количество документов)")

# 7.11 Разделение по возрастным группам
if 'age_years' in X_train.columns:
    bins = [0, 25, 35, 45, 55, 65, 100]
    labels = ['<25', '25-35', '35-45', '45-55', '55-65', '65+']
    X_train['age_group'] = pd.cut(X_train['age_years'], bins=bins, labels=labels, include_lowest=True)
    X_val['age_group'] = pd.cut(X_val['age_years'], bins=bins, labels=labels, include_lowest=True)
    print("Создан признак: age_group (возрастные группы)")

# 7.12 Временные признаки
time_features = ['registration_timestamp', 'publication_timestamp', 'last_phone_number_change']
for feature in time_features:
    if feature in X_train.columns:
        # Категоризация по времени: недавно, умеренно давно, давно
        # Определяем границы на основе квантилей
        q33 = X_train[feature].quantile(0.33)
        q66 = X_train[feature].quantile(0.66)
        
        def categorize_time(x):
            if pd.isna(x):
                return 'unknown'
            elif x <= q33:
                return 'recent'
            elif x <= q66:
                return 'moderate'
            else:
                return 'long_ago'
        
        X_train[f'{feature}_category'] = X_train[feature].apply(categorize_time)
        X_val[f'{feature}_category'] = X_val[feature].apply(categorize_time)
        print(f"Создан категориальный признак для: {feature}")

# 8. Кодирование категориальных признаков
print("\n" + "="*70)
print("8. КОДИРОВАНИЕ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ")
print("="*70)

# Обновленный список категориальных признаков после feature engineering
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
print(f"Количество категориальных признаков для кодирования: {len(categorical_features)}")
print("Список категориальных признаков:", categorical_features)

# 8.1 Кодирование признаков с низкой кардинальностью (One-Hot Encoding)
low_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() < 10]
print(f"\nПризнаки с низкой кардинальностью (<10 уникальных значений): {len(low_cardinality_cols)}")
print(low_cardinality_cols)

# 8.2 Кодирование признаков с высокой кардинальностью (Target Encoding или Frequency Encoding)
high_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() >= 10]
print(f"Признаки с высокой кардинальностью (>=10 уникальных значений): {len(high_cardinality_cols)}")
print(high_cardinality_cols)

# One-Hot Encoding для признаков с низкой кардинальностью
if low_cardinality_cols:
    # Удаляем исходные категориальные признаки и добавляем закодированные
    X_train_encoded = pd.get_dummies(X_train[low_cardinality_cols], prefix=low_cardinality_cols, drop_first=True)
    X_val_encoded = pd.get_dummies(X_val[low_cardinality_cols], prefix=low_cardinality_cols, drop_first=True)
    
    # Выравниваем столбцы в случае, если в валидационной выборке меньше категорий
    X_train_encoded, X_val_encoded = X_train_encoded.align(X_val_encoded, join='left', axis=1, fill_value=0)
    
    # Добавляем закодированные признаки к основным данным
    X_train = pd.concat([X_train.drop(low_cardinality_cols, axis=1), X_train_encoded], axis=1)
    X_val = pd.concat([X_val.drop(low_cardinality_cols, axis=1), X_val_encoded], axis=1)
    
    print(f"Применен One-Hot Encoding для {len(low_cardinality_cols)} признаков")
    print(f"Размер данных после кодирования: {X_train.shape}")

# Target Encoding для признаков с высокой кардинальностью
for col in high_cardinality_cols:
    if col in X_train.columns:
        print(f"\nПрименение Target Encoding для признака: {col}")
        
        # Проверяем, есть ли в признаке категории, которых нет в обучающей выборке
        train_categories = set(X_train[col].unique())
        val_categories = set(X_val[col].unique())
        new_categories = val_categories - train_categories
        
        if new_categories:
            print(f"  Обнаружены новые категории в валидационной выборке: {new_categories}")
        
        # Вычисляем среднее значение target для каждой категории на обучающих данных
        category_means = y_train.groupby(X_train[col]).mean()
        global_mean = y_train.mean()
        
        # Заполняем пропущенные категории глобальным средним
        category_means = category_means.fillna(global_mean)
        
        # Добавляем шум для уменьшения переобучения
        noise_scale = 0.01 * category_means.std()
        category_means_noisy = category_means + np.random.normal(0, noise_scale, size=len(category_means))
        
        # Применяем mapping к обучающей и валидационной выборкам
        X_train[f'{col}_target_enc'] = X_train[col].map(category_means_noisy).fillna(global_mean)
        X_val[f'{col}_target_enc'] = X_val[col].map(category_means_noisy).fillna(global_mean)
        
        # Удаляем исходный категориальный признак
        X_train.drop(col, axis=1, inplace=True)
        X_val.drop(col, axis=1, inplace=True)
        
        print(f"  Target Encoding для {col} успешно применен")

# Проверка после кодирования категориальных признаков
remaining_categorical = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
if remaining_categorical:
    print(f"\nВНИМАНИЕ: В данных остались следующие категориальные признаки: {remaining_categorical}")
    print("Рекомендуется проверить процесс кодирования или удалить эти признаки")
    
    # Дополнительное кодирование оставшихся категориальных признаков
    for col in remaining_categorical:
        if col in X_train.columns:
            print(f"Применение Label Encoding для оставшегося категориального признака: {col}")
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            
            # Проверяем тип данных признака и преобразуем его при необходимости
            if pd.api.types.is_categorical_dtype(X_train[col]):
                # Если это категориальный тип, преобразуем в строковый
                X_train[col] = X_train[col].astype(str)
                X_val[col] = X_val[col].astype(str)
                print(f"  Признак {col} преобразован из категориального типа в строковый")
            
            # Заполняем пропуски перед кодированием
            X_train[col] = X_train[col].fillna('Missing')
            X_val[col] = X_val[col].fillna('Missing')
            
            # Применяем Label Encoding
            X_train[col] = le.fit_transform(X_train[col])
            X_val[col] = le.transform(X_val[col])
            
            print(f"  Label Encoding для {col} успешно применен")

# 9. Масштабирование числовых признаков
print("\n" + "="*70)
print("9. МАСШТАБИРОВАНИЕ ЧИСЛОВЫХ ПРИЗНАКОВ")
print("="*70)

# Обновленный список числовых признаков после кодирования и feature engineering
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Количество числовых признаков для масштабирования: {len(numeric_features)}")

# Определяем признаки для разных типов масштабирования
# Оставляем некоторые признаки без масштабирования
exclude_features = ['target', 'reco_id_curr', 'children_count', 'family_members__count']
exclude_features += [col for col in numeric_features if '_flag' in col.lower()]

# Признаки для RobustScaler (чувствительные к выбросам)
financial_features = ['income', 'loan_body', 'annuity_payment', 'goods_price']
ratio_features = ['loan_to_income_ratio', 'payment_to_income_ratio', 'goods_to_income_ratio', 'payment_burden']
experience_features = ['work_experience_years', 'work_experience_to_age_ratio']

robust_features = [f for f in numeric_features if f in financial_features + ratio_features + experience_features]
robust_features = [f for f in robust_features if f in X_train.columns and f not in exclude_features]

# Признаки для StandardScaler (остальные числовые признаки)
standard_features = [f for f in numeric_features if f not in robust_features + exclude_features]

print(f"Признаки для RobustScaler: {len(robust_features)}")
print(f"Признаки для StandardScaler: {len(standard_features)}")
print(f"Признаки без масштабирования: {len(exclude_features)}")

# Масштабирование с помощью RobustScaler
if robust_features:
    robust_scaler = RobustScaler()
    X_train[robust_features] = robust_scaler.fit_transform(X_train[robust_features])
    X_val[robust_features] = robust_scaler.transform(X_val[robust_features])
    print("Применен RobustScaler для финансовых признаков и отношений")

# Масштабирование с помощью StandardScaler
if standard_features:
    standard_scaler = StandardScaler()
    X_train[standard_features] = standard_scaler.fit_transform(X_train[standard_features])
    X_val[standard_features] = standard_scaler.transform(X_val[standard_features])
    print("Применен StandardScaler для остальных числовых признаков")

# 10. Отбор признаков (опционально)
print("\n" + "="*70)
print("10. ОТБОР ПРИЗНАКОВ")
print("="*70)

# Получаем только числовые признаки для вычисления корреляции
numeric_features_for_corr = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

if numeric_features_for_corr:
    print(f"Количество числовых признаков для анализа корреляции: {len(numeric_features_for_corr)}")
    
    # Удаляем признаки с очень высокой корреляцией между собой (>0.9)
    corr_matrix = X_train[numeric_features_for_corr].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr_features = [column for column in upper.columns if any(upper[column] > 0.9)]
    
    if high_corr_features:
        print(f"Найдено признаков с высокой корреляцией (>0.9): {len(high_corr_features)}")
        print("Удаляем следующие признаки:", high_corr_features)
        X_train.drop(high_corr_features, axis=1, inplace=True)
        X_val.drop(high_corr_features, axis=1, inplace=True)
    else:
        print("Признаков с очень высокой корреляцией (>0.9) не найдено")
else:
    print("Нет числовых признаков для анализа корреляции")

# 11. Итоговый обзор предобработанных данных
print("\n" + "="*70)
print("11. ИТОГОВЫЙ ОБЗОР ПРЕДОБРАБОТАННЫХ ДАННЫХ")
print("="*70)

print(f"\nФинальная размерность обучающей выборки: {X_train.shape}")
print(f"Финальная размерность валидационной выборки: {X_val.shape}")

# Проверка на пропущенные значения
missing_train_final = X_train.isnull().sum().sum()
missing_val_final = X_val.isnull().sum().sum()
print(f"\nОставшиеся пропуски в обучающей выборке: {missing_train_final}")
print(f"Оставшиеся пропуски в валидационной выборке: {missing_val_final}")

# Проверка типов данных
print("\nТипы данных в обучающей выборке:")
print(X_train.dtypes.value_counts())

# Статистика по числовым признакам
print("\nСтатистика по числовым признакам (обучающая выборка):")
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
if numeric_cols:
    stats_df = pd.DataFrame({
        'mean': X_train[numeric_cols].mean(),
        'std': X_train[numeric_cols].std(),
        'min': X_train[numeric_cols].min(),
        'max': X_train[numeric_cols].max()
    })
    print(stats_df.head(10).round(4))

# 12. Сохранение предобработанных данных
print("\n" + "="*70)
print("12. СОХРАНЕНИЕ ПРЕДОБРАБОТАННЫХ ДАННЫХ")
print("="*70)

# Добавляем целевую переменную обратно в обучающие данные для сохранения
train_processed = X_train.copy()
train_processed['target'] = y_train

val_processed = X_val.copy()
val_processed['target'] = y_val

# Сохраняем предобработанные данные
train_processed.to_csv('train_processed.csv', index=False)
val_processed.to_csv('val_processed.csv', index=False)

print("Предобработанные данные успешно сохранены:")
print(f"- train_processed.csv: {train_processed.shape}")
print(f"- val_processed.csv: {val_processed.shape}")

# Также сохраняем информацию о препроцессинге для использования в продакшене
import joblib

# Создаем объект с информацией о препроцессинге
preprocessing_info = {
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'robust_features': robust_features,
    'standard_features': standard_features,
    'high_corr_features_removed': high_corr_features,
    'feature_engineering': [
        'loan_to_income_ratio', 'payment_to_income_ratio', 'age_years', 
        'work_experience_years', 'combined_external_score', 'total_bki_requests'
    ]
}

joblib.dump(preprocessing_info, 'preprocessing_info.pkl')
print("\nИнформация о препроцессинге сохранена в preprocessing_info.pkl")

print("\nПредобработка данных завершена!")

Загрузка данных...
Размер обучающей выборки: (261384, 122)

3. РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ВАЛИДАЦИОННУЮ ВЫБОРКИ
Размер обучающей выборки: (209107, 120)
Размер валидационной выборки: (52277, 120)

Распределение целевой переменной в обучающей выборке:
target
0   91.9238
1    8.0762
Name: proportion, dtype: float64

Распределение целевой переменной в валидационной выборке:
target
0   91.9238
1    8.0762
Name: proportion, dtype: float64

Всего числовых признаков: 104
Всего категориальных признаков: 16

5. ОБНАРУЖЕНИЕ И ОБРАБОТКА ВЫБРОСОВ ВО ВСЕХ ЧИСЛОВЫХ ПРИЗНАКАХ
Начало комплексной обработки выбросов...
Всего числовых признаков для обработки: 104

6. ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
Количество признаков с пропусками в обучающей выборке: 97

Топ-10 признаков с наибольшим процентом пропусков:
non_living_apartments_avg    72.3089
non_living_apartments_medi   72.2377
non_living_apartments_mode   72.0205
mode_commonarea              71.0464
median_commonarea            71.0459
average_com

In [10]:
# 1. Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# 2. Загрузка данных
print("Загрузка данных...")
df = pd.read_csv('data.csv')

print(f"Размер обучающей выборки: {df.shape}")
print(f"Размер тестовой выборки: {df.shape if 'test_df' in locals() else 'Не загружена'}")

# 3. Разделение на train/validation с сохранением дисбаланса классов
print("\n" + "="*70)
print("3. РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ВАЛИДАЦИОННУЮ ВЫБОРКИ")
print("="*70)

# Удаляем идентификаторы, которые не нужны для обучения
if 'reco_id_curr' in df.columns:
    df.drop(['reco_id_curr'], axis=1, inplace=True)

# Разделяем признаки и целевую переменную
X = df.drop('target', axis=1)
y = df['target']

# Стратифицированное разделение для сохранения дисбаланса классов
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,  # Сохраняем распределение классов
    random_state=42
)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер валидационной выборки: {X_val.shape}")

# Проверка распределения целевой переменной
print("\nРаспределение целевой переменной в обучающей выборке:")
print(y_train.value_counts(normalize=True) * 100)
print("\nРаспределение целевой переменной в валидационной выборке:")
print(y_val.value_counts(normalize=True) * 100)

# 4. Базовое заполнение пропущенных значений
print("\n" + "="*70)
print("4. БАЗОВОЕ ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
print("="*70)

# Определение типов признаков
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Количество числовых признаков: {len(numeric_features)}")
print(f"Количество категориальных признаков: {len(categorical_features)}")

# Анализ пропусков
missing_train = X_train.isnull().mean() * 100
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)
print("\nТоп-10 признаков с наибольшим процентом пропусков в обучающей выборке:")
print(missing_train.head(10))

# 4.1 Заполнение пропусков в числовых признаках (медианой)
numeric_imputer = SimpleImputer(strategy='median')
X_train[numeric_features] = numeric_imputer.fit_transform(X_train[numeric_features])
X_val[numeric_features] = numeric_imputer.transform(X_val[numeric_features])
print("Заполнение пропусков в числовых признаках (медианой) выполнено")

# 4.2 Заполнение пропусков в категориальных признаках (модой)
if categorical_features:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X_train[categorical_features] = categorical_imputer.fit_transform(X_train[categorical_features])
    X_val[categorical_features] = categorical_imputer.transform(X_val[categorical_features])
    print("Заполнение пропусков в категориальных признаках (модой) выполнено")

# Проверка оставшихся пропусков
remaining_missing_train = X_train.isnull().sum().sum()
remaining_missing_val = X_val.isnull().sum().sum()
print(f"\nОставшиеся пропуски в обучающей выборке: {remaining_missing_train}")
print(f"Оставшиеся пропуски в валидационной выборке: {remaining_missing_val}")

# 5. Базовое кодирование категориальных признаков
print("\n" + "="*70)
print("5. БАЗОВОЕ КОДИРОВАНИЕ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ")
print("="*70)

# Обновленный список категориальных признаков после заполнения пропусков
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
print(f"Количество категориальных признаков для кодирования: {len(categorical_features)}")

# 5.1 One-Hot Encoding для признаков с низкой кардинальностью
if categorical_features:
    # Определяем признаки с низкой кардинальностью (<10 уникальных значений)
    low_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() < 10]
    high_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() >= 10]
    
    print(f"Признаки с низкой кардинальностью (<10 уникальных значений): {len(low_cardinality_cols)}")
    print(f"Признаки с высокой кардинальностью (>=10 уникальных значений): {len(high_cardinality_cols)}")
    
    # One-Hot Encoding для признаков с низкой кардинальностью
    if low_cardinality_cols:
        X_train_encoded = pd.get_dummies(X_train[low_cardinality_cols], prefix=low_cardinality_cols, drop_first=True)
        X_val_encoded = pd.get_dummies(X_val[low_cardinality_cols], prefix=low_cardinality_cols, drop_first=True)
        
        # Выравниваем столбцы
        X_train_encoded, X_val_encoded = X_train_encoded.align(X_val_encoded, join='left', axis=1, fill_value=0)
        
        # Добавляем закодированные признаки к основным данным
        X_train = pd.concat([X_train.drop(low_cardinality_cols, axis=1), X_train_encoded], axis=1)
        X_val = pd.concat([X_val.drop(low_cardinality_cols, axis=1), X_val_encoded], axis=1)
        
        print(f"One-Hot Encoding применен для {len(low_cardinality_cols)} признаков")
    
    # Label Encoding для признаков с высокой кардинальностью
    for col in high_cardinality_cols:
        if col in X_train.columns:
            print(f"Label Encoding для признака: {col}")
            le = LabelEncoder()
            
            # Преобразуем в строки и заполняем пропуски
            X_train[col] = X_train[col].astype(str)
            X_val[col] = X_val[col].astype(str)
            
            # Обучаем и применяем кодировщик
            X_train[col] = le.fit_transform(X_train[col])
            X_val[col] = le.transform(X_val[col])

# 6. Базовое масштабирование числовых признаков
print("\n" + "="*70)
print("6. БАЗОВОЕ МАСШТАБИРОВАНИЕ ЧИСЛОВЫХ ПРИЗНАКОВ")
print("="*70)

# Обновленный список числовых признаков после кодирования
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Количество числовых признаков для масштабирования: {len(numeric_features)}")

# Исключаем целевую переменную и идентификаторы, если они остались
exclude_features = ['target', 'reco_id_curr']
numeric_features = [f for f in numeric_features if f not in exclude_features]

# Применяем StandardScaler для всех числовых признаков
if numeric_features:
    scaler = StandardScaler()
    X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
    X_val[numeric_features] = scaler.transform(X_val[numeric_features])
    print(f"StandardScaler применен для {len(numeric_features)} числовых признаков")

# 7. Итоговый обзор обработанных данных
print("\n" + "="*70)
print("7. ИТОГОВЫЙ ОБЗОР ОБРАБОТАННЫХ ДАННЫХ")
print("="*70)

print(f"\nФинальная размерность обучающей выборки: {X_train.shape}")
print(f"Финальная размерность валидационной выборки: {X_val.shape}")

# Проверка на пропущенные значения
missing_train_final = X_train.isnull().sum().sum()
missing_val_final = X_val.isnull().sum().sum()
print(f"\nОставшиеся пропуски в обучающей выборке: {missing_train_final}")
print(f"Оставшиеся пропуски в валидационной выборке: {missing_val_final}")

# Проверка типов данных
print("\nТипы данных в обучающей выборке:")
print(X_train.dtypes.value_counts())

# Статистика по числовым признакам
print("\nСтатистика по числовым признакам (обучающая выборка):")
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
if numeric_cols:
    stats_df = pd.DataFrame({
        'mean': X_train[numeric_cols].mean(),
        'std': X_train[numeric_cols].std(),
        'min': X_train[numeric_cols].min(),
        'max': X_train[numeric_cols].max()
    })
    print(stats_df.head(10).round(4))

# 8. Сохранение обработанных данных
print("\n" + "="*70)
print("8. СОХРАНЕНИЕ ОБРАБОТАННЫХ ДАННЫХ")
print("="*70)

# Добавляем целевую переменную обратно в обучающие данные для сохранения
train_processed = X_train.copy()
train_processed['target'] = y_train

val_processed = X_val.copy()
val_processed['target'] = y_val

# Сохраняем обработанные данные
train_processed.to_csv('train_minimal_preprocessing.csv', index=False)
val_processed.to_csv('val_minimal_preprocessing.csv', index=False)

print("Обработанные данные успешно сохранены:")
print(f"- train_minimal_preprocessing.csv: {train_processed.shape}")
print(f"- val_minimal_preprocessing.csv: {val_processed.shape}")

Загрузка данных...
Размер обучающей выборки: (261384, 122)
Размер тестовой выборки: Не загружена

3. РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ВАЛИДАЦИОННУЮ ВЫБОРКИ
Размер обучающей выборки: (209107, 120)
Размер валидационной выборки: (52277, 120)

Распределение целевой переменной в обучающей выборке:
target
0   91.9238
1    8.0762
Name: proportion, dtype: float64

Распределение целевой переменной в валидационной выборке:
target
0   91.9238
1    8.0762
Name: proportion, dtype: float64

4. БАЗОВОЕ ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
Количество числовых признаков: 104
Количество категориальных признаков: 16

Топ-10 признаков с наибольшим процентом пропусков в обучающей выборке:
median_commonarea            69.8355
mode_commonarea              69.8355
average_commonarea           69.8355
non_living_apartments_mode   69.3989
non_living_apartments_medi   69.3989
non_living_apartments_avg    69.3989
fondkapremon_mode            68.3621
average_living_apartments    68.3368
median_living_apartments     68.